In [9]:
import os
import pandas as pd
import numpy as np
import urllib
import seaborn as sns
import matplotlib.pyplot as plt
import calendar
%matplotlib inline

In [10]:
path = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\normals\anomalies\temperature_anomalies'

avgt_figpath = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\normals\anomalies\temperature_anomalies\figures\monthly\COOP-CoAg'
mint_figpath = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\minimums\anomalies\temperature_anomalies\figures\monthly\COOP-CoAg'
maxt_figpath = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\maximums\anomalies\temperature_anomalies\figures\monthly\COOP-CoAg'

avgt_tabpath = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\normals\anomalies\temperature_anomalies\tables\monthly\COOP-CoAg'
mint_tabpath = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\minimums\anomalies\temperature_anomalies\tables\monthly\COOP-CoAg'
maxt_tabpath = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\maximums\anomalies\temperature_anomalies\tables\monthly\COOP-CoAg'

os.chdir(path)

xl_file = r"C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\colocation_metadata.xlsx"
csv = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\normals\anomalies\temperature_anomalies\vessel.csv'

In [11]:
xl = pd.ExcelFile(xl_file)

meta = xl.parse('metadata')
meta['sdate'] = meta.sdate.dt.date
meta['edate'] = meta.edate.dt.date
meta.drop(meta.columns[3],axis=1)

,ID,sdate,edate,COOP_ID,distance (miles)
0,AKR02,1992-07-01,2018-12-31,50109,0.020000
1,ALT01,1992-03-17,2018-12-31,53006,15.938171
2,AVN01,1992-06-04,2018-12-31,50839,11.482940
3,BNV01,2010-10-12,2018-12-31,51071,0.476716
4,BRG01,2002-07-31,2018-12-31,50945,2.813569
5,BRK01,2013-11-07,2018-12-31,56320,6.661099
6,BRL02,1992-01-02,2018-12-31,51121,8.599777
7,BRL03,2008-03-21,2018-12-31,51121,4.083030
8,CBL01,2015-05-08,2018-12-31,53359,12.241012
9,CKP01,2014-07-25,2018-12-31,58690,2.280432


In [12]:
n=0
for index,rows in meta.iterrows():
    try:
        print('Grabbing CoAg data for ' + str(meta.ID[n]))
        urllib.request.urlretrieve(
            'http://coagmet.colostate.edu/cgi-bin/web_services.pl?type=daily&sids=' + str(meta.ID[n]) + '&sdate=' + str(meta.sdate[n]) + '&edate=' + str(meta.edate[n]) + '&elems=tave,tmax,tmin',
            filename=csv)
        coag = pd.read_csv(csv)
        coag = coag.reset_index()
    except Exception as e:
        print('Something went wrong while calling the CoAg data for ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Cleaning CoAg data for ' + str(meta.ID[n]))
        coag.columns.values[0] = 'station'
        coag.columns.values[1] = 'date'
        coag.columns.values[2] = 'CoAg_tmean'
        coag.columns.values[3] = 'CoAg_tmax'
        coag.columns.values[4] = 'CoAg_tmin'

        coag['date'] = pd.to_datetime(coag.date)
        coag['julian'] = coag.date.dt.strftime('%j')

        coag.CoAg_tmean = coag.CoAg_tmean.astype(float)
        coag.CoAg_tmax = coag.CoAg_tmax.astype(float)
        coag.CoAg_tmin = coag.CoAg_tmin.astype(float)

        coag.CoAg_tmean = 32 + (coag.CoAg_tmean * (9/5))
        coag.CoAg_tmax = 32 + (coag.CoAg_tmax * (9/5))
        coag.CoAg_tmin = 32 + (coag.CoAg_tmin * (9/5))

        coag=coag[:-1]
    except Exception as e:
        print('Something went wrong while cleaning the CoAg data for ' + str(meta.ID[n]) + ': ' + e.args[0])

    try:

        print('Grabbing COOP data for ' + str(meta.ID[n]))
        urllib.request.urlretrieve(
            'http://data.rcc-acis.org/StnData?sid=0' + str(meta.COOP_ID[n]) + '&sdate=' + str(meta.sdate[n]) + '&edate=' + str(meta.edate[n]) + '&elems=maxt,mint,avgt&output=csv',
            filename=csv)

        coop = pd.read_csv(csv)
        coop = coop.reset_index()
    except Exception as e:
        print('Something went wrong while calling the COOP data for ' + str(meta.ID[n]) + ': ' + e.args[0])

    try:
        print('Cleaning COOP data for ' + str(meta.ID[n]))
        coop.columns.values[0] = 'date'
        coop.columns.values[1] = 'COOP_tmax'
        coop.columns.values[2] = 'COOP_tmin'
        coop.columns.values[3] = 'COOP_avgt'

        coop = coop.replace('M',np.nan,regex=True)
        coop = coop.replace('T', 0.005)

        coop.COOP_tmax = coop.COOP_tmax.astype(float)
        coop.COOP_tmin = coop.COOP_tmin.astype(float)
        coop.COOP_avgt = coop.COOP_avgt.astype(float)

        coop = coop.replace('M',np.nan,regex=True)
        coop = coop.replace('T', 0.005)

        coop['date'] = pd.to_datetime(coag.date)
        coop['julian'] = coag.date.dt.strftime('%j')
    except Exception as e:
        print('Something went wrong while cleaning the COOP data for ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Calculating COOP normals for ' + str(meta.ID[n]))
        normal = coop.groupby(['julian']).mean().reset_index()
    except Exception as e:
        print('Something went wrong while calculating COOP normals for ' + str(meta.ID[n]) + ': ' + e.args[0])

    try:
        print('Merging COOP and CoAg data for ' + str(meta.ID[n]))
        anomaly = coag.merge(normal,'outer',left_on='julian',right_on='julian')
    except Exception as e:
        print('Something went wrong while merging the datasets for ' + str(meta.ID[n]) + ': ' + e.args[0])

    try:
        print('Cleaning merged dataset for ' + str(meta.ID[n]))
        avgt_only = ['CoAg_tmax','CoAg_tmin','COOP_tmax','COOP_tmin']
        maxt_only = ['CoAg_tmean','COOP_avgt','CoAg_tmin','COOP_tmin']
        mint_only = ['CoAg_tmean','COOP_avgt','CoAg_tmax','COOP_tmax']

        avgt_anomaly = anomaly.drop(avgt_only,axis=1)
        maxt_anomaly = anomaly.drop(maxt_only,axis=1)
        mint_anomaly = anomaly.drop(mint_only,axis=1)

        avgt_anomaly = anomaly[['date','julian','CoAg_tmean','COOP_avgt']]    
        maxt_anomaly = anomaly[['date','julian','CoAg_tmax','COOP_tmax']]
        mint_anomaly = anomaly[['date','julian','CoAg_tmin','COOP_tmin']]

        avgt_anomaly['anomaly'] = avgt_anomaly['CoAg_tmean'] - avgt_anomaly['COOP_avgt']
        maxt_anomaly['anomaly'] = maxt_anomaly['CoAg_tmax'] - maxt_anomaly['COOP_tmax']
        mint_anomaly['anomaly'] = mint_anomaly['CoAg_tmin'] - mint_anomaly['COOP_tmin']

        avgt_anomaly['date'] = pd.to_datetime(avgt_anomaly.date)
        maxt_anomaly['date'] = pd.to_datetime(maxt_anomaly.date)
        mint_anomaly['date'] = pd.to_datetime(mint_anomaly.date)

        avgt_anomaly['month'] = avgt_anomaly.date.dt.month
        maxt_anomaly['month'] = maxt_anomaly.date.dt.month
        mint_anomaly['month'] = mint_anomaly.date.dt.month


        ma_avgt_anomaly = avgt_anomaly.groupby(['month']).mean().reset_index()
        ma_maxt_anomaly = maxt_anomaly.groupby(['month']).mean().reset_index()
        ma_mint_anomaly = mint_anomaly.groupby(['month']).mean().reset_index()
        
        ma_avgt_anomaly['month'] = ma_avgt_anomaly['month'].apply(lambda x: calendar.month_abbr[x])
        ma_maxt_anomaly['month'] = ma_maxt_anomaly['month'].apply(lambda x: calendar.month_abbr[x])
        ma_mint_anomaly['month'] = ma_mint_anomaly['month'].apply(lambda x: calendar.month_abbr[x])
    except Exception as e:
        print('Something went wrong while cleaning the merged data set for ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Storing anomaly tables for ' + str(meta.ID[n]))
        ma_avgt_anomaly.to_csv(avgt_tabpath + r'\COOP-CoAg_' + str(meta.ID[n]) + '_avgt_monthly_anomaly.csv')
        ma_maxt_anomaly.to_csv(maxt_tabpath + r'\COOP-CoAg_' + str(meta.ID[n]) + '_maxt_monthly_anomaly.csv')
        ma_mint_anomaly.to_csv(mint_tabpath + r'\COOP-CoAg_' + str(meta.ID[n]) + '_mint_monthly_anomaly.csv')
    except Exception as e:
        print('Something went wrong while storing tables for ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Visualizing and storing anomaly figures for ' + str(meta.ID[n]))
        avgt_ax = sns.catplot(x='month',y='anomaly',data=ma_avgt_anomaly,kind='bar',height=8.27,aspect=11.7/8.27)
        plt.savefig(avgt_figpath + r'\COOP-CoAg_' + str(meta.ID[n]) + '_ma_avgt_anomaly.png')
        plt.close()
        maxt_ax = sns.catplot(x='month',y='anomaly',data=ma_maxt_anomaly,kind='bar',height=8.27,aspect=11.7/8.27)
        plt.savefig(maxt_figpath + r'\COOP-CoAg_' + str(meta.ID[n]) + '_ma_maxt_anomaly.png')
        plt.close()
        mint_ax = sns.catplot(x='month',y='anomaly',data=ma_mint_anomaly,kind='bar',height=8.27,aspect=11.7/8.27)
        plt.savefig(mint_figpath + r'\COOP-CoAg_' + str(meta.ID[n]) + '_ma_mint_anomaly.png')
        plt.close()
    except Exception as e:
        print('Something went wrong while visualizing and storing the anomaly data for ' + str(meta.ID[n]) + ': ' + e.args[0])     
    n=n+1


Grabbing CoAg data for AKR02
Cleaning CoAg data for AKR02
Grabbing COOP data for AKR02
Cleaning COOP data for AKR02
Calculating COOP normals for AKR02
Merging COOP and CoAg data for AKR02
Cleaning merged dataset for AKR02
Storing anomaly tables for AKR02
Visualizing and storing anomaly figures for AKR02


C:\Users\student\Anaconda3\lib\site-packages\ipykernel_launcher.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\student\Anaconda3\lib\site-packages\ipykernel_launcher.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\student\Anaconda3\lib\site-packages\ipykernel_launcher.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

Grabbing CoAg data for ALT01
Cleaning CoAg data for ALT01
Grabbing COOP data for ALT01
Cleaning COOP data for ALT01
Calculating COOP normals for ALT01
Merging COOP and CoAg data for ALT01
Cleaning merged dataset for ALT01
Storing anomaly tables for ALT01
Visualizing and storing anomaly figures for ALT01
Grabbing CoAg data for AVN01
Cleaning CoAg data for AVN01
Grabbing COOP data for AVN01
Cleaning COOP data for AVN01
Something went wrong while cleaning the COOP data for AVN01: index 2 is out of bounds for axis 0 with size 2
Calculating COOP normals for AVN01
Something went wrong while calculating COOP normals for AVN01: julian
Merging COOP and CoAg data for AVN01
Cleaning merged dataset for AVN01
Storing anomaly tables for AVN01
Visualizing and storing anomaly figures for AVN01
Grabbing CoAg data for BNV01
Cleaning CoAg data for BNV01
Grabbing COOP data for BNV01
Cleaning COOP data for BNV01
Calculating COOP normals for BNV01
Merging COOP and CoAg data for BNV01
Cleaning merged dataset

Cleaning COOP data for HLY01
Calculating COOP normals for HLY01
Merging COOP and CoAg data for HLY01
Cleaning merged dataset for HLY01
Storing anomaly tables for HLY01
Visualizing and storing anomaly figures for HLY01
Grabbing CoAg data for HLY02
Cleaning CoAg data for HLY02
Grabbing COOP data for HLY02
Cleaning COOP data for HLY02
Calculating COOP normals for HLY02
Merging COOP and CoAg data for HLY02
Cleaning merged dataset for HLY02
Storing anomaly tables for HLY02
Visualizing and storing anomaly figures for HLY02
Grabbing CoAg data for HNE01
Cleaning CoAg data for HNE01
Grabbing COOP data for HNE01
Cleaning COOP data for HNE01
Calculating COOP normals for HNE01
Merging COOP and CoAg data for HNE01
Cleaning merged dataset for HNE01
Storing anomaly tables for HNE01
Visualizing and storing anomaly figures for HNE01
Grabbing CoAg data for HOT01
Cleaning CoAg data for HOT01
Grabbing COOP data for HOT01
Cleaning COOP data for HOT01
Calculating COOP normals for HOT01
Merging COOP and CoAg

Cleaning COOP data for PKH01
Calculating COOP normals for PKH01
Merging COOP and CoAg data for PKH01
Cleaning merged dataset for PKH01
Storing anomaly tables for PKH01
Visualizing and storing anomaly figures for PKH01
Grabbing CoAg data for PKR01
Cleaning CoAg data for PKR01
Grabbing COOP data for PKR01
Cleaning COOP data for PKR01
Calculating COOP normals for PKR01
Merging COOP and CoAg data for PKR01
Cleaning merged dataset for PKR01
Storing anomaly tables for PKR01
Visualizing and storing anomaly figures for PKR01
Grabbing CoAg data for PNR01
Cleaning CoAg data for PNR01
Grabbing COOP data for PNR01
Cleaning COOP data for PNR01
Calculating COOP normals for PNR01
Merging COOP and CoAg data for PNR01
Cleaning merged dataset for PNR01
Storing anomaly tables for PNR01
Visualizing and storing anomaly figures for PNR01
Grabbing CoAg data for RFD01
Cleaning CoAg data for RFD01
Grabbing COOP data for RFD01
Cleaning COOP data for RFD01
Calculating COOP normals for RFD01
Merging COOP and CoAg